<a href='https://www.darshan.ac.in/'> <img src='https://www.darshan.ac.in/Content/media/DU_Logo.svg' width="250" height="300"/></a>
<pre>
<center><b><h1>Data Mining</b></center>
<center><b><h1>Lab - 10</b></center>    
<center><b><h1>Keval Ajani</b></center>
<center><b><h1>Enrollment No. : 23010101006</b></center>
<center><b><h1>Roll No. : 186</b></center>        
<pre>



# Implement Decision Tree(ID3) in python
Uses Information Gain to choose the best feature to split. 

Recursively builds the tree until stopping conditions are met.

1) Calculate Entropy for the dataset.<BR>
2) Calculate Information Gain for each feature. <BR>
3) Choose the feature with maximum Information Gain. <BR>
4) Split dataset into subsets for that feature. <BR>
5) Repeat recursively until: <BR>

All samples in a node have the same label.<BR>
No features are left.<BR>
No data is left.

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

##  import Pandas, Numpy

In [1]:
import pandas as pd
import numpy as np

##  Create Following Data

In [29]:
data = pd.DataFrame({
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
})

In [30]:
data

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


## Now Define Function to  Calculate Entropy

In [19]:
def entropy(y):
  value , counts = np.unique(y,return_counts=True)
  # print(f"Values: {value}, Counts: {counts}")
  probabilities = counts / counts.sum()
  # print(f"Probabilities: {probabilities}")
  return -np.sum(probabilities * np.log2(probabilities))

entropy(data['PlayTennis'])

np.float64(0.9402859586706311)

## Testing of Above Function - 
y = np.array(['Yes', 'No',   'Yes', 'Yes'])  
Function Call - > entropy(y))

output - 0.8112781244591328

In [15]:
y = np.array(['Yes','No','Yes','Yes'])
print(entropy(y))

0.8112781244591328


## Define function to Calculate Information Gain

In [23]:
def information_gain(data, split_attribute, target):
    total_entropy = entropy(data[target])
    value,counts = np.unique(data[split_attribute], return_counts=True)
    weighted_entropy = 0
    for i in range(len(value)):
        subset = data[data[split_attribute] == value[i]]
        subset_entropy = entropy(subset[target])
        weighted_entropy += (counts[i] / counts.sum()) * subset_entropy
    return total_entropy - weighted_entropy

## Testing of Above Function- 
data = pd.DataFrame({
    'Weather': ['Sunny', 'Sunny', 'Rain', 'Rain'],
    'Play':    ['Yes', 'No',   'Yes', 'Yes']
})

Function Call - > information_gain(data, 'Weather', 'Play')


Output - 0.31127812445913283

In [27]:
test_data = pd.DataFrame({
    'Weather': ['Sunny', 'Sunny', 'Rain', 'Rain'],
    'Play':    ['Yes', 'No',   'Yes', 'Yes']
})
print(information_gain(test_data, 'Weather', 'Play'))

0.31127812445913283


## Implement ID3 Algo

In [33]:
data['PlayTennis'].mode()

0    Yes
Name: PlayTennis, dtype: object

In [39]:
def id3(data, features, target):
    # If all labels are same → return the label
    if len(np.unique(data[target])) == 1:
        return np.unique(data[target])[0]  
    
    # If no features left → return majority label
    if len(features) == 0:
        return data[target].mode()[0]   # The mode is the value that appears most often. There can be multiple modes.Always returns Series even if only one value is returned.
    
    # Choose best feature
    gains = [information_gain(data, feature, target) for feature in features]
    best_feature = features[np.argmax(gains)]
    
    tree = {best_feature: {}}
    
    # For each value of best feature → branch
    for value in np.unique(data[best_feature]):
        sub_data = data[data[best_feature] == value].drop(columns =[best_feature])
        if len(sub_data) == 0:
            tree[best_feature][value] = data[target].mode()[0]
        else:
            subtree = id3(sub_data, [f for f in features if f != best_feature], target)
            tree[best_feature][value] = subtree
    return tree

## Use ID3

In [42]:
target = 'PlayTennis'
features = [f for f in list(data.columns) if f != target]
tree = id3(data,features,target)

## Print Tree

In [51]:
tree

{'Outlook': {'Overcast': 'Yes',
  'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
  'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

## Extra: Create Predict Function

In [45]:
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree  # If tree is a leaf node, return the prediction
    
    feature = next(iter(tree))  # Get the first key of the tree
    feature_value = sample[feature]
    
    if feature_value in tree[feature]:
        return predict(tree[feature][feature_value], sample)
    else:
        return None  # If the feature value is not found in the tree

##  Extra: Predict for a sample
sample = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}


Your Answer ?

In [47]:
sample = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}
prediction = predict(tree, sample)
print(f"Prediction : {prediction}")

Prediction : No
